In [ ]:
# Script to create date for Date Dimesion Landing

In [ ]:
# Import required libraries
import sys
from lib.spark_session import get_spark_session
from lib.utils import date_data, get_string_cols
from lib.job_control import insert_log, get_max_timestamp
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import current_timestamp
from datetime import datetime
from delta import DeltaTable

In [ ]:
# JOB Parameters
rundate = "20220101"
schema_name = "edw_ld"
table_name = "dim_date_ld"
table_full_name = f"{schema_name}.{table_name}" 

In [ ]:
# Generate Spark Session
spark: SparkSession = get_spark_session(f"Landing load - {table_full_name}")
print("SPARK_APP: Spark UI - " + spark.sparkContext.uiWebUrl)

In [ ]:
# Spark Configs
spark.conf.set("spark.sql.shuffle.partitions", 8)

In [ ]:
# Define cols for date landing
_cols = ["date", "day", "month", "year", "day_of_month"]
_data = date_data(rundate, 2)

In [ ]:
# Create Raw dataframe and get col info
df_raw = spark.createDataFrame(data=_data, schema=_cols)
print("SPARK_APP: Printing Raw Schema --")
df_raw.printSchema()

# Get Landing count
print("SPARK_APP: Landing data count - " + str(df_raw.count()))

In [ ]:
# Cast all columns to String
df_casted = df_raw.selectExpr(get_string_cols(spark, df_raw))
print("SPARK_APP: Casted all columns to String")

# Add audit columns
df_ld = df_casted.withColumn("insert_dt", current_timestamp())
print("SPARK_APP: Added AUDIT column")

# Get Final Layer count
print("SPARK_APP: Final layer data count - " + str(df_ld.count()))
print("SPARK_APP: Printing Landing Schema --")
df_ld.printSchema()

In [ ]:
# Write the data to landing layer checking if table exists
if get_max_timestamp(spark, schema_name, table_name) != "1900-01-01 00:00:00.000000":
    df_ld.write \
        .format("delta") \
        .mode("append") \
        .saveAsTable(table_full_name)
else:
    df_ld.write \
        .format("delta") \
        .saveAsTable(table_full_name)

print("SPARK_APP: Data written to landing layer")

In [ ]:
# Add job details in JOB CONTROL
insert_log(spark, schema_name, table_name, datetime.now())
print("SPARK_APP: Update JOB Control Log")

In [ ]:
spark.sql(f"select * from edw.job_control where table_name = '{table_name}' order by insert_dt desc limit 1").show(truncate=False)

In [ ]:
# Generate Symlink manifest for Athena Access
dt = DeltaTable.forName(spark, table_full_name)
dt.generate("symlink_format_manifest")
print("SPARK_APP: Symlink Manifest file generated")

In [ ]:
spark.stop()